In [ ]:

!pip install pinecone voyageai


In [ ]:
!pip install protobuf==4.23.4 --force-reinstall


In [17]:
import pandas as pd
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm
import numpy as np
from urllib.parse import urlparse
import requests
import pandas as pd
import voyageai
import PIL
from pinecone import Pinecone
import json


In [ ]:

df = pd.read_json("super_dry_women.json", lines=True)


In [ ]:
df.head()

In [ ]:
# Importing ENGLISH_STOP_WORDS within the same cell
def extract_keywords(text):
    words=text.split()
    keywords = [word for word in words if word not in ENGLISH_STOP_WORDS]
    return " ".join(keywords)
def concatenate(row):

    return (

        str(row["product_name"]) + " " +
        str(row["product_description"]) + " " +
        str(row["color"])+ " "+
        str(row["product_size"])+" "+
        str(row["product_material"])+" "+
        str(row["product_price"])
    )
df["text_blob"] = df.apply(concatenate, axis=1)

df["keywords"] = df["text_blob"].apply(lambda x: extract_keywords(x))


In [ ]:
pids=["P_"+str(i) for i in range(1,df.shape[0]+1)]

df["P_ID"]=pids

In [ ]:
df_exploded = df.explode('images')
df_exploded.reset_index(drop=True, inplace=True)

In [ ]:
def download_image(url):
    if pd.isna(url) or url == '':
        print("None")
        return None
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        print(f"Skipping invalid URL: {url}")
        return Image.new('RGB', (224, 224), color='white')

    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            img = Image.open(response.raw).convert("RGB")
            print("Downloaded"
            )
            return img
        else:
            print(f"Skipping URL with status code {response.status_code}: {url}")
            return Image.new('RGB', (224, 224), color='white')
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")
        return Image.new('RGB', (224, 224), color='white')

In [ ]:


vo = voyageai.Client(api_key="pa-int4C_9EqFIwO4MusvoIGdIUE0t0jHP3E8YX5V6xXyB")
# This will automatically use the environment variable VOYAGE_API_KEY.
# Alternatively, you can use vo = voyageai.Client(api_key="<your secret key>")

# Example input containing a text string and PIL image object
def create_embeddings(row):
  inputs = [
      [row["keywords"], download_image(row["images"])]
  ]

  # Vectorize inputs
  result = vo.multimodal_embed(inputs, model="voyage-multimodal-3")
  return result.embeddings


In [ ]:
df_exploded["embeddings"]=df_exploded.apply(create_embeddings, axis=1)

In [ ]:
df_exploded["embeddings"]=df_exploded["embeddings"].apply(lambda x: x[0])

In [ ]:
eids=["E_"+str(i) for i in range(1,df_exploded.shape[0]+1)]
eids

In [ ]:
df_exploded["E_ID"]=eids

In [ ]:
df_exploded.to_json("Downloads/superdry_women_embeddings.json", orient="records",lines=True)


# Use from here if you already have embeddings

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm
import numpy as np
from urllib.parse import urlparse
import requests
import pandas as pd
import voyageai
import PIL
from pinecone import Pinecone
import json
from collections import defaultdict

In [27]:
df=pd.read_json("Downloads/superdry_women_embeddings.json",lines=True)

In [19]:

pc = Pinecone(api_key="pcsk_3JkiG3_Bd8FvZB1VaeUgFHdkw9BAHadPLqFRzCuk46qSx6iA7SvFEA93hwASkpoHPFxBUB")
index_name = "productdisc"

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         vector_type="dense",
#         dimension=1024,

#         metric="cosine",
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         ),
#         deletion_protection="disabled",
#         tags={
#             "environment": "development"
#         }
#     )

In [21]:
index = pc.Index("productdisc")

In [23]:
#index.delete(delete_all=True, namespace="multimodal")

'index.delete(delete_all=True, namespace="image")'

In [29]:

def store_embeddings(df, index):
    vectors = []


    for _, row in df.iterrows():
        uid = str(row["E_ID"])

        if row.get("embeddings") is not None:
              metadata = {
                      "product_id": row.get("P_ID"),
                      "title": row.get("product_name"),
                      "description": row.get("product_description"),
                      "Material": row.get("product_material"),
                      "color": row.get("color"),
                      "Size": row.get("product_size"),
                      "image_url": row.get("images"),
                      "price": row.get("product_price"),
                      "keywords": row.get("keywords")
                  }

              vectors.append({
                "id": f"{uid}",
                "values": row["embeddings"],
                "metadata": metadata
            })

   
    batch_size = 100
    for i in range(0, len(vectors), batch_size):
        index.upsert(vectors=vectors[i:i+batch_size], namespace="multimodal")
   


In [31]:
store_embeddings(df, index)

In [39]:
vo = voyageai.Client(api_key="pa-int4C_9EqFIwO4MusvoIGdIUE0t0jHP3E8YX5V6xXyB")

In [65]:
query = "cardigans under $80"
query_embed=vo.multimodal_embed([[query]], model="voyage-multimodal-3").embeddings
results = index.query(
    vector=query_embed[0],
    top_k=df.shape[0],
    namespace="multimodal",
    include_metadata=True
)
scores_text=defaultdict(list)
for match in results['matches']:
    scores_text[match['metadata']["keywords"]].append(match['score'])
for key, value in scores_text.items():
    scores_text[key]=sum(value)/len(value)
top_k = dict(sorted(scores_text.items(), key=lambda x: x[1], reverse=True)[:30])
documents=list(top_k.keys())
documents = documents

reranking = vo.rerank(query, documents, model="rerank-2", top_k=10)
for r in reranking.results:
    print(f"Document: {r.document}")
    print(f"Relevance Score: {r.relevance_score}")
    print()

Document: Slouchy Cable Roll Neck Jumper Loose Fit – comfort meets cool, stylish loose cut makes must-have shape Roll neck long sleeves Chunky cable knit design Ribbed cuffs hem Superdry tab sleeve Offering timeless style comfort, Slouchy Cable Roll Neck Jumper designed fit seamlessly wardrobe. The raglan sleeves contribute perfectly relaxed shape chic cable knit pattern looks effortlessly trend, culminating truly essential piece colder months. Color: Dark Charcoal Grey Model: Height 5ft 9.5in. Chest 32in Model wearing: UK10, EU38, US6 Material: Cotton 100% Bleach: Do Not Bleach Dry: Do Not Tumble Dry Dry Clean: Do Not Dry Clean Wash: Machine Wash - Cold (30°C) Price: $74.95
Relevance Score: 0.72265625

Document: Slouchy Stitch Roll Neck Knit Loose Fit – comfort meets cool, stylish loose cut makes must-have shape Roll neck Knitted Ribbed hem, neckline, cuffs Contrast sewn-in Superdry tag Refresh winter wardrobe Slouchy Stitch Roll Neck Knitted Jumper. Crafted loose fit, it’s trendy cla

In [ ]:
df.head(30)["description"].to_list()

In [ ]:
#pa-int4C_9EqFIwO4MusvoIGdIUE0t0jHP3E8YX5V6xXyB

# Method 2: Average the embeddings 

In [70]:
import pandas as pd
import numpy as np

df_agg=df.copy()

# Convert embeddings to np.array
df_agg['embedding'] = df_agg['embeddings'].apply(np.array)

# Drop 'eid' as it's not needed
df_agg = df_agg.drop(columns=['E_ID'])

# Group by pid and aggregate
df_agg = df_agg.groupby('P_ID').agg({
    'embeddings': lambda x: np.mean(np.stack(x), axis=0),
    'images': list,
    'product_name': 'first',
    "product_description":"first",
    "color":"first",# assuming same across rows
    'product_size': 'first',
    "product_material":"first",
    "product_price":"first",
    "keywords":"first"# assuming same across rows
}).reset_index()


In [72]:
df_agg

,P_ID,embeddings,images,product_name,product_description,color,product_size,product_material,product_price,keywords
0,P_1,"[0.050488281260000004, 0.02949218752, 0.031640...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Casual Linen Boyfriend Shirt,"Loose Fit – where comfort meets cool, a stylis...",Color: Blue wide stripe,"Loose Fit – where comfort meets cool, a stylis...",Material:\nLinen 100%\nBleach:\nDo Not Bleach\...,Price: $84.95,Casual Linen Boyfriend Shirt Loose Fit – comfo...
1,P_10,"[0.018600463875, -0.0011422633875, -0.00147207...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Cargo Shorts,Relaxed fit – the classic Superdry fit. Not to...,Color: Olive Night Green,Model: Height 5ft 8.5in. Chest 31.5in\nModel w...,Trim:\nCotton 100%\nMaterial:\nCotton 100%\nBl...,Price: $69.95,Cargo Shorts Relaxed fit – classic Superdry fi...
2,P_100,"[0.07430013021666666, 0.011530558266666666, 0....",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Tattoo Rhinestone Fitted T-Shirt,Slim fit – designed to fit closer to the body ...,Color: Fig Purple,Model: Height 5ft 10in. Chest 32in\nModel wear...,Material:\nCotton 100%\nBleach:\nDo Not Bleach...,Price: $59.95,Tattoo Rhinestone Fitted T-Shirt Slim fit – de...
3,P_101,"[0.04170735675, -0.016377766933333333, 0.01783...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Embroidered Core Logo Boxy Loose T-Shirt,"Boxy fit – looser and more flowing, for those ...",Color: Ash Rose,Model: Height 5ft 8.5in. Chest 31.5in\nModel w...,Material:\nCotton 100%\n\nDry:\nDo Not Tumble ...,Price: $39.95,Embroidered Core Logo Boxy Loose T-Shirt Boxy ...
4,P_102,"[0.07116699215, 0.017349243150000002, 0.052246...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Vintage Logo Rhinestone Slim T-Shirt,Slim fit – designed to fit closer to the body ...,Color: Silver Blue,Slim fit – designed to fit closer to the body ...,Material:\nCotton 100%\nBleach:\nDo Not Bleach...,Price: $44.95,Vintage Logo Rhinestone Slim T-Shirt Slim fit ...
...,...,...,...,...,...,...,...,...,...,...
942,P_95,"[0.06461588541666667, 0.007501324016666667, 0....",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Chore Jacket,Relaxed fit – the classic Superdry fit. Not to...,Color: OFF WHITE,Relaxed fit – the classic Superdry fit. Not to...,Lining:\nCotton 100%\nTrim:\nCotton 100%\nMate...,Price: $104.95,Chore Jacket Relaxed fit – classic Superdry fi...
943,P_96,"[0.0591796875, -0.0019737243600000003, 0.04912...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Embellished Varsity Relaxed T-Shirt,Relaxed fit – the classic Superdry fit. Not to...,Color: DARK GREY MARL,Model: Height 5ft 9.5in. Chest 30.5in\nModel w...,Material:\nOrganic Cotton 100%\nBleach:\nDo No...,Price: $39.95,Embellished Varsity Relaxed T-Shirt Relaxed fi...
944,P_97,"[0.05456542968333333, -0.0108718872, 0.0336914...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Chore Jacket,Relaxed fit – the classic Superdry fit. Not to...,Color: Ash Rose,Relaxed fit – the classic Superdry fit. Not to...,Lining:\nCotton 100%\nTrim:\nCotton 100%\nMate...,Price: $104.95,Chore Jacket Relaxed fit – classic Superdry fi...
945,P_98,"[0.0495605469, 0.009012858066666666, 0.0068155...",[https://images.laguna-live.sd.co.uk/zoom/uplo...,Studios Hoodie,Relaxed fit – the classic Superdry fit. Not to...,,Relaxed fit – the classic Superdry fit. Not to...,"Material:\nRecycled Polyester 45%, Cotton 55%\...",Price: $74.95,Studios Hoodie Relaxed fit – classic Superdry ...


In [74]:
def store_embeddings(df, index):
    vectors = []


    for _, row in df.iterrows():
        uid = str(row["P_ID"])

        if row.get("embeddings") is not None:
              metadata = {
                      
                      "title": row.get("product_name"),
                      "description": row.get("product_description"),
                      "Material": row.get("product_material"),
                      "color": row.get("color"),
                      "Size": row.get("product_size"),
                      "image_url": row.get("images"),
                      "price": row.get("product_price"),
                      "keywords": row.get("keywords")
                  }

              vectors.append({
                "id": f"{uid}",
                "values": row["embeddings"],
                "metadata": metadata
            })

        # if row.get("image_embedding") is not None:
        #     trimmed_image_meta = safe_trim_metadata(row)
        #     image_vectors.append({
        #         "id": f"{uid}",
        #         "values": row["image_embedding"],
        #         "metadata": trimmed_image_meta
        #     })
    batch_size = 100
    for i in range(0, len(vectors), batch_size):
        index.upsert(vectors=vectors[i:i+batch_size], namespace="multimodal_AVG")
    # for i in range(0, len(image_vectors), batch_size):
    #     index.upsert(vectors=image_vectors[i:i+batch_size], namespace="image")


In [76]:
store_embeddings(df_agg, index)

In [144]:
query = "cardigans under 80 above height 177cm"
query_embed=vo.multimodal_embed([[query]], model="voyage-multimodal-3").embeddings
results = index.query(
    vector=query_embed[0],
    top_k=20,
    namespace="multimodal_AVG",
    include_metadata=True
    )
documents=[result["metadata"]["keywords"] for result in results["matches"]]
ids = [match["id"] for match in results["matches"]]

reranking = vo.rerank(query, documents, model="rerank-2", top_k=10)
for r in reranking.results:
    print(f"Document: {r.document}")
    print(f"Relevance Score: {r.relevance_score}")
    


Document: Slouchy Cable Roll Neck Jumper Loose Fit – comfort meets cool, stylish loose cut makes must-have shape Roll neck long sleeves Chunky cable knit design Ribbed cuffs hem Superdry tab sleeve Offering timeless style comfort, Slouchy Cable Roll Neck Jumper designed fit seamlessly wardrobe. The raglan sleeves contribute perfectly relaxed shape chic cable knit pattern looks effortlessly trend, culminating truly essential piece colder months. Color: Dark Charcoal Grey Model: Height 5ft 9.5in. Chest 32in Model wearing: UK10, EU38, US6 Material: Cotton 100% Bleach: Do Not Bleach Dry: Do Not Tumble Dry Dry Clean: Do Not Dry Clean Wash: Machine Wash - Cold (30°C) Price: $74.95
Relevance Score: 0.55859375
Document: Knitted Rugby Half Zip Jumper Loose Fit – comfort meets cool, stylish loose cut makes must-have shape High collar half-zip fastening Cable knit design Ribbed cuffs hemline Signature tabs sleeve hemline Designed bring touch luxury knitwear, Knitted Rugby Half Zip Jumper features

In [134]:
reranked_ids

['P_468',
 'P_382',
 'P_722',
 'P_465',
 'P_368',
 'P_367',
 'P_429',
 'P_439',
 'P_352',
 'P_137']